In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [2]:
#Parameters
samplenum = 50
epochs = 20
hiddenlayers = [120]
input_size = 3
output_size = 3
learning_rate = 0.01
time_length = 60; #seconds

In [3]:
# Generate simulation
dyn = d.PyDyn('Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

In [4]:
#Sample targets only variables in z direction
y_target = np.zeros((samplenum,3))
y_target[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
y_target[:,1] = 2
y_target= torch.tensor(y_target)

## Building the custon Simulation Activation Function

In [6]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        state = dyn.compute(p)
        y_pred = torch.tensor(state.y[-3:])
        print(f'y_pred: {y_pred.shape}')
        
        ctx.save_for_backward(input)
        
        return y_pred
    
    @staticmethod
    def backward(ctx, grad_output):
        print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        state= dyn.compute(p)
        dy_dp = dyn.dy_dp(state, p)
        dy_dp = dy_dp[-3:, :]
        print(f'shape of dy/dp: {dy_dp.shape}')
        #print(f'shape of grad_output: {grad_output.shape}')
        grad_output = grad_output.unsqueeze(0)
        
        grad_input = torch.tensor(dy_dp).t().mm(grad_output.t()).t()
        print(f'shape of dgrad_input: {grad_input.shape}')
        return grad_input

Simulate = Simulate.apply

## Building the Model

In [7]:
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, hiddenlayers[0])
        self.H1 = nn.Linear(hiddenlayers[0], 3*time_length)
        #self.H1 = nn.Linear(hiddenlayers[0], hiddenlayers[1])
        #self.H2 = nn.Linear(hiddenlayers[1], 3*time_length)
        self.P = nn.Linear(3*time_length, 3*time_length)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        #x = self.H2(x)
        #x = self.Relu(x)
        x = self.P(x)
        return x


model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

y_target = y_target.float()

## Train the model

In [8]:
torch.autograd.set_detect_anomaly(True)

start_time = time.time()
weight_c1 = 1 # p start condition
weight_c2 = 1 # p smoothness condition
losses = []
y_preds= np.zeros((samplenum, 3))
p_preds= np.zeros((samplenum, 3*time_length))

#y_pred = torch.tensor(y_pred)
for i in range(epochs):
    for s in range(samplenum):
        y_truth = y_target[s, :]
        p_pred = model(y_truth)
        y_pred = Simulate(p_pred)
        y_preds[s, :] = y_pred.detach()
        p_preds[s, :] = p_pred.detach()
        smoothness_error = sum((p_pred[0:3*(time_length-1)] - p_pred[3:3*time_length])**2)
        #loss = torch.sqrt(criterion(y_pred.float(), y_truth)) # RMSE
        #loss = criterion(y_pred.float(), y_truth) + weight_c1*(sum(p_pred[0:3]-torch.tensor(dyn.p_init[0:3])))**2  # MSE + start condition penalty + p smoothness condition penalty
        loss = criterion(y_pred.float(), y_truth) + weight_c1*criterion(p_pred[0:3].double(), torch.tensor(dyn.p_init[0:3])) + weight_c2*smoothness_error# MSE + start condition penalty + p smoothness condition penalty
        #loss =sum((y_pred.float()-y_truth)**2) 
        losses.append(loss)
        optimizer.zero_grad()
        #Back Prop
        loss.backward()
        optimizer.step()
    print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}  smoothness error: {smoothness_error.item():10.8f}')
    i+=1

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed

rch.Size([180])
y_pred: torch.Size([3])
torch.Size([3])
shape of dy/dp: (3, 180)
shape of dgrad_input: torch.Size([1, 180])
input: torch.Size([180])
y_pred: torch.Size([3])
torch.Size([3])
shape of dy/dp: (3, 180)
shape of dgrad_input: torch.Size([1, 180])
input: torch.Size([180])
y_pred: torch.Size([3])
torch.Size([3])
shape of dy/dp: (3, 180)
shape of dgrad_input: torch.Size([1, 180])
input: torch.Size([180])
y_pred: torch.Size([3])
torch.Size([3])
shape of dy/dp: (3, 180)
shape of dgrad_input: torch.Size([1, 180])
input: torch.Size([180])
y_pred: torch.Size([3])
torch.Size([3])
shape of dy/dp: (3, 180)
shape of dgrad_input: torch.Size([1, 180])
input: torch.Size([180])
y_pred: torch.Size([3])
torch.Size([3])
shape of dy/dp: (3, 180)
shape of dgrad_input: torch.Size([1, 180])
input: torch.Size([180])
y_pred: torch.Size([3])
torch.Size([3])
shape of dy/dp: (3, 180)
shape of dgrad_input: torch.Size([1, 180])
input: torch.Size([180])
y_pred: torch.Size([3])
torch.Size([3])
shape of dy/d

KeyboardInterrupt: 

In [10]:
print(y_pred.grad)

None


In [8]:
#Save Model

if len(losses) == epochs*(samplenum):
    torch.save(model.state_dict(), 'Trained_Model_300420_300s_100e_onlyZpos.pt')
    print('Model saved')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

Model saved


## Test forward propagation

In [65]:
y_target= torch.tensor([0, 2, 0.5])
p = model(y_target)
p = p.detach().numpy()

y_pred_state = dyn.compute(p)
y_pred = y_pred_state.y[-3:]

print(y_pred)
print(y_target)
print(f'p trajecory: {p}')
print(f"y trajectory: {y_pred_state.y}")
error = 0
for i in range(time_length-1):
    step = sum((p[3*i:3*i+3] - p[3*i+3:3*i+6])**2)
    error = error + step
print(error/time_length)


[-0.05843178  1.91289042  0.5277091 ]
tensor([0.0000, 2.0000, 0.5000])
p trajecory: [-0.3475796   2.6794724  -0.25029546 -0.3169132   2.5978498  -0.34320584
 -0.32316363  2.5009358  -0.35768396 -0.326442    2.3999605  -0.3670242
 -0.33319473  2.2966313  -0.36000848 -0.334164    2.1921992  -0.35824865
 -0.340296    2.0910745  -0.3528615  -0.33155766  1.9894711  -0.3444112
 -0.32445875  1.8836269  -0.33305183 -0.31845888  1.7835768  -0.3210051
 -0.31766537  1.6858059  -0.3096674  -0.3099547   1.5929787  -0.2992336
 -0.3000655   1.4961368  -0.2928601  -0.2948266   1.4039501  -0.28120518
 -0.28426725  1.3143059  -0.26231772 -0.28116924  1.2280066  -0.25244734
 -0.27570817  1.1476697  -0.23181176 -0.2700998   1.0675237  -0.21655297
 -0.2676601   0.99025464 -0.20319659 -0.26645288  0.917227   -0.18590827
 -0.27453178  0.8673251  -0.16666439 -0.26827368  0.80920917 -0.15555471
 -0.27674037  0.7628604  -0.13407066 -0.2836355   0.726278   -0.11686522
 -0.2898554   0.7004952  -0.09889606 -0.2900

In [51]:
test = torch.tensor(1,1,1,2,2,2,3,3,3,4,4,4)
p_pred[0:3*(time_length-1)] - p_pred[3:3*time_length])**2)

array([], dtype=float32)

## Torch Script Conversion and Saving

In [36]:
input_example = torch.tensor(y_target[0,:])
traced_script_module = torch.jit.trace(model, input_example)
original = model(test_input)

# Test the torch script
test_input= torch.tensor([0, 2, 0.5])
output_example = traced_script_module(test_input)
print(output_example[-12:])
print(original)

tensor([1.9574, 0.0000, 0.0000, 2.7701, 0.0000], grad_fn=<SliceBackward>)


In [ ]:
# Save serialized model
traced_script_module.save("CPP_example_model_latest.pt")